# Proses Crawling API Twitter




# Import Packages
Packages meliput library yang mendukung dalam proses crawling. pada penelitian ini digunakan library package tweepy. . Tweepy adalah salah datu library packages open source untuk mendapatkan data tweet. Tweepy menyertakan satu set kelas dan metode yang mewakili model Twitter dan titik akhir API. 

Library lain untuk proses preprocessing, menghitung ekstraksi fitur tf-idf, menghitung menggunakan algortima multinomial naive bayes digunakan pada penelitian ini untuk memudahkan proses klasifikasi dan mengevaluasi model

In [ ]:
import tweepy as tw
import pandas as pd

In [ ]:
#Masukkan twitter token API
api_key = "HTBlthMxY3hRikPeYqWp1lzqz"
api_key_secret = "2jFAVjmRaF9j2K8k8gK3TjQKjkbrNSCa5NCO0vuJXOS0yM3eN3"
access_token = "748381377796644864-ZvR8BzKpANo0e8cnNxVodXyJ1WAhmu4"
access_token_secret = "hIxjoSk7mex2aDduDOMRq8H3DZpt8Vpf62BBlWUaJqyij"

In [ ]:
# melakukan authentication atau registrasi api key ke api twitter
auth = tw.OAuthHandler(api_key, api_key_secret)
# Mendapat akses dari token yang didapat
auth.set_access_token(access_token, access_token_secret)
# memanggil API twitter
api = tw.API(auth)

In [ ]:
# Mulai crawling search tweet
tweets = tw.Cursor(api.search_tweets, 
q="jabatan tiga periode OR masa jabatan presiden OR presiden tiga periode",
lang="id", tweet_mode='extended', since="01-07-2022", until="30-12-2022").items(600)

In [ ]:
for teks in tweets:
    #Membuat penyimpanan 
    teks_properties= {}
    #Membuat properti tanggal, nama pengguna dan isi tweet.
    teks_properties["tanggal"]=teks.created_at
    teks_properties["pengguna"]=teks.user.screen_name
    teks_properties["isi teks"]= teks.full_text

In [ ]:
#Menyimpan dataset ke file penyimpanan

df = pd.DataFrame(tweets)
df.to_csv('dataset_jabatan_presiden.csv')


In [ ]:
df = pd.read_csv('dataset_jabatan_presiden.csv')
df[:10]

,VALUE SENTIMEN,tanggal,pengguna,isi teks
0,0,2022-05-16T10:02:28.000Z,Fraksi Gerindra MPR/DPR Republik Indonesia,"""Kondisi bangsa ini menghadapi kenyataan bahwa..."
1,0,2022-06-14T06:47:02.000Z,Putra Jembes,"""Tiga periode itu proyek mimpi. Saat ini Jokow..."
2,0,2022-06-29T09:21:57.000Z,Hadi Djawas,"""Tolak penundaan pemilu dan masa perpangjang j..."
3,0,2022-06-14T11:39:58.000Z,cah lokal jiwa nasional,#BacktoBDM @bdm2502 bersama Ulin Yusron (@ulin...
4,0,2022-06-29T01:02:53.000Z,Answerts,#Projo #Jokowi #3Periode \n\nSaya bilang ke te...
5,0,2022-06-29T06:03:06.000Z,SuTaryo,#Rilis | Partai Gelora: Parpol Pro Jokowi Munc...
6,0,2022-04-22T15:51:42.000Z,Junet,#salam3periode\n#rakyatmenghendaki\n#SuaraRaky...
7,0,2022-06-15T16:05:02.000Z,Repelita Online,..https://t.co/sDQsz9zkJk\n\nHotman Paris Huta...
8,0,2022-06-11T23:59:46.000Z,Ayy-SAGA,@_Qirani__Ayara @jokowi Kalau menurut konfirma...
9,0,2022-06-10T13:02:06.000Z,Santriwati,@03__nakula Saya kok jadi berkobar untuk menyu...


# Proses *Preprocessing*

proses preprocessing meliputi data cleansing, case folding, tokenizing/Normalization, stopword/filtering, dan stemming

Proses Data cleansing

In [ ]:
import re
#Membuat fungsi untuk data cleansing
def data_cleansing(text):
  #Menghilangkan username
  text = re.sub(r'@[^\s]+','',text)
  #Menghapus link 
  text = re.sub(r'http?://\S+|www\.\S+','',text)
  #Menghapus karakter dan angka
  text = re.sub(r'[-+]?[0-9]+','',text)
  text = re.sub(r'[^\w\s]','',text)
  text = text.strip()
  return text

Hasil Data Cleansing

In [ ]:
contoh_sample = df['isi teks'].iloc[3]
datacleansing = data_cleansing(contoh_sample)

print('contoh sampel\t: ', contoh_sample)
print('data cleansing\t: ', datacleansing)

contoh sampel	:  #BacktoBDM @bdm2502 bersama Ulin Yusron (@ulinyusron) membahas isu-isu tentang penundaan pemilu 2024. Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode. Tonton video di sini: https://t.co/tVSTaXx97A https://t.co/zB52Un4cYl
data cleansing	:  BacktoBDM  bersama Ulin Yusron  membahas isuisu tentang penundaan pemilu  Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode Tonton video di sini httpstcotVSTaXxA httpstcozBUncYl


Proses Case Folding
Berdasarkan NLP by Stanford case folding menjadi lower text atau menjadi kata kecil karena memiliki string matching

In [ ]:
import string
#fungsi mengubah huruf kapital menjadi huruf biasa
def case_folding(text):
  text = text.lower()
  return text

Hasil Case Folding

In [ ]:
contoh_sample = df['isi teks'].iloc[3]
datacleansing = data_cleansing(contoh_sample)
casefolding = case_folding(datacleansing)

print('contoh sampel\t: ', contoh_sample)
print('data cleansing\t: ', datacleansing)
print('case folding\t: ', casefolding)

contoh sampel	:  #BacktoBDM @bdm2502 bersama Ulin Yusron (@ulinyusron) membahas isu-isu tentang penundaan pemilu 2024. Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode. Tonton video di sini: https://t.co/tVSTaXx97A https://t.co/zB52Un4cYl
data cleansing	:  BacktoBDM  bersama Ulin Yusron  membahas isuisu tentang penundaan pemilu  Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode Tonton video di sini httpstcotVSTaXxA httpstcozBUncYl
case folding	:  backtobdm  bersama ulin yusron  membahas isuisu tentang penundaan pemilu  ulin yusron yang seorang relawan jokowi dengan tegas menolak jabatan presiden tiga periode tonton video di sini httpstcotvstaxxa httpstcozbuncyl


Proses Normalization dan Tokenizing

In [ ]:
!wget https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv

--2023-05-03 23:21:39--  https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80969 (79K) [text/plain]
Saving to: ‘key_norm.csv’

key_norm.csv        100%[===================>]  79.07K  --.-KB/s    in 0.01s   

2023-05-03 23:21:39 (5.99 MB/s) - ‘key_norm.csv’ saved [80969/80969]



In [ ]:
key_normal = pd.read_csv('key_norm.csv')
print(key_normal.head())

key_normal.shape

   _id      singkat       hasil
0    1         abis       habis
1    2       accent     tekanan
2    3       accept      terima
3    4     accident  kecelakaan
4    5  achievement    prestasi


(3720, 3)

In [ ]:
def text_normalize(text):
   text = ' '.join([key_normal[key_normal['singkat'] == word]['hasil'].values[0] 
      if (key_normal['singkat'] == word).any() 
      else word for word in text.split()])
   text = str.lower(text) 
   return text

In [ ]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

def tokenizing(text):
  text = word_tokenize(text)
  return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
contoh_sample = df['isi teks'].iloc[3]
datacleansing = data_cleansing(contoh_sample)
casefolding = case_folding(datacleansing)
token = tokenizing(casefolding)

print('contoh sampel\t: ', contoh_sample)
print('data cleansing\t: ', datacleansing)
print('case folding\t: ', casefolding)
print('tokenizing\t: ', token)


contoh sampel	:  #BacktoBDM @bdm2502 bersama Ulin Yusron (@ulinyusron) membahas isu-isu tentang penundaan pemilu 2024. Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode. Tonton video di sini: https://t.co/tVSTaXx97A https://t.co/zB52Un4cYl
data cleansing	:  BacktoBDM  bersama Ulin Yusron  membahas isuisu tentang penundaan pemilu  Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode Tonton video di sini httpstcotVSTaXxA httpstcozBUncYl
case folding	:  backtobdm  bersama ulin yusron  membahas isuisu tentang penundaan pemilu  ulin yusron yang seorang relawan jokowi dengan tegas menolak jabatan presiden tiga periode tonton video di sini httpstcotvstaxxa httpstcozbuncyl
tokenizing	:  ['backtobdm', 'bersama', 'ulin', 'yusron', 'membahas', 'isuisu', 'tentang', 'penundaan', 'pemilu', 'ulin', 'yusron', 'yang', 'seorang', 'relawan', 'jokowi', 'dengan', 'tegas', 'menolak', 'jabatan', 'presiden', 'tiga', 'periode', 'ton

Proses Stopword

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
list_stopword = stopwords.words('indonesian')
list_stopword

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

# Mengetahui new_stopwordlist ada atau belum

In [ ]:
# mengetahui panjang stopword indonesia
panjang_word = len(list_stopword)
print(f"Jumlah total stopword adalah = {panjang_word}")


In [ ]:
#menambahkan stopword baru manual.
#kemudian dicek di list_stopword apakah ada atau tidak.

while True:
    # menginputkan kata baru
    list_baru = input("Masukkan kata stopword list baru dan ketik 'selesai' untuk berhenti input ")
    # berhenti looping
    if list_baru == "selesai":
      break

    # mengecek apakah semua elemen sudah ada di list lama
    if list_baru in list_stopword:
        print("Kata yang diinputkan sudah ada di stopword lama")
        break
    else:
        list_stopword.append(list_baru)
        print("Kata baru telah ditambahkan ke stopword lama")

print("List stopword: ", list_stopword)

Masukkan kata stopword list baru dan ketik 'selesai' untuk berhenti input gacor
Kata baru telah ditambahkan ke stopword lama
Masukkan kata stopword list baru dan ketik 'selesai' untuk berhenti input tidak
Kata yang diinputkan sudah ada di stopword lama
List stopword:  ['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benar

In [ ]:
#menambahkan stopwordlist
stopword_baru = ["yg", "dg", "rt", "dgn", "ny", "d",
'klo', 'kalo', 'amp', 'biar', 'bikin','kaa','rt', '&amp', 'yah',
'bilang', 'gak', 'ga', 'krn', 'nya', 'sdh', 'aja', 'n', 't', 'nyg', 'nih', 'sih', 'si', 
'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn',
'hehe', 'pen', 'u', 'nan', 'loh', 'y', 'org','gub','unt','kalm','ngetuit']
list_stopword = list_stopword + stopword_baru
list_stopword.remove('tiga')


Definisi Kelas Stopwords

In [ ]:
def stop_words(text):
  data_bersih = [] #membuat list kosong
  text = text.split() #membagi teks
  for word in text:
    if word not in list_stopword:
        clean_words.append(word)
  return" ".join(data_bersih)

Hasil Stopwords

In [ ]:
contoh_sample = df['isi teks'].iloc[3]
datacleansing = data_cleansing(contoh_sample)
casefolding = case_folding(datacleansing)
stopwords = stop_words(casefolding)

print('contoh sampel\t: ', contoh_sample)
print('data cleansing\t: ', datacleansing)
print('case folding\t: ', casefolding)
print('stopwords\t: ', stopwords)

contoh sampel	:  #BacktoBDM @bdm2502 bersama Ulin Yusron (@ulinyusron) membahas isu-isu tentang penundaan pemilu 2024. Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode. Tonton video di sini: https://t.co/tVSTaXx97A https://t.co/zB52Un4cYl
data cleansing	:  BacktoBDM  bersama Ulin Yusron  membahas isuisu tentang penundaan pemilu  Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode Tonton video di sini
case folding	:  backtobdm  bersama ulin yusron  membahas isuisu tentang penundaan pemilu  ulin yusron yang seorang relawan jokowi dengan tegas menolak jabatan presiden tiga periode tonton video di sini
stopwords	:  backtobdm ulin yusron membahas isuisu penundaan pemilu ulin yusron relawan jokowi menolak jabatan presiden tiga periode tonton video


Proses Stemming

In [ ]:
pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 11.5 MB/s eta 0:00:00


In [ ]:
#mengimport library stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# buat stem
factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemming(text):
  text = stemmer.stem(text)
  return text


Hasil Stemming

In [ ]:
contoh_sample = df['isi teks'].iloc[3]
datacleansing = data_cleansing(contoh_sample)
casefolding = case_folding(datacleansing)
stopwords = stop_words(casefolding)
stem = stemming(stopwords)

print('contoh sampel\t: ', contoh_sample)
print('data cleansing\t: ', datacleansing)
print('case folding\t: ', casefolding)
print('stopwords\t: ', stopwords)
print('stemming\t: ', stem)

contoh sampel	:  #BacktoBDM @bdm2502 bersama Ulin Yusron (@ulinyusron) membahas isu-isu tentang penundaan pemilu 2024. Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode. Tonton video di sini: https://t.co/tVSTaXx97A https://t.co/zB52Un4cYl
data cleansing	:  BacktoBDM  bersama Ulin Yusron  membahas isuisu tentang penundaan pemilu  Ulin Yusron yang seorang relawan Jokowi dengan tegas menolak jabatan presiden tiga periode Tonton video di sini
case folding	:  backtobdm  bersama ulin yusron  membahas isuisu tentang penundaan pemilu  ulin yusron yang seorang relawan jokowi dengan tegas menolak jabatan presiden tiga periode tonton video di sini
stopwords	:  backtobdm ulin yusron membahas isuisu penundaan pemilu ulin yusron relawan jokowi menolak jabatan presiden tiga periode tonton video
stemming	:  backtobdm ulin yusron bahas isuisu tunda milu ulin yusron rawan jokowi tolak jabat presiden tiga periode tonton video


In [ ]:
#preprocessing pipeline
def text_preprocessing_proses(text):
    text = data_cleansing(text)
    text = case_folding(text)
    text = text_normalize(text)
    text = stop_words(text)
    text = stemming(text)
    return text

In [ ]:
df['data_bersih'] = df['isi teks'].apply(text_preprocessing_proses)
df.to_csv('dataset_bersih.csv')
df[:5][['isi teks','data_bersih']]

,isi teks,data_bersih
0,"""Kondisi bangsa ini menghadapi kenyataan bahwa...",kondisi bangsa hadap nyata hormat jokowi bench...
1,"""Tiga periode itu proyek mimpi. Saat ini Jokow...",tiga periode proyek mimpi jokowi hadap sadar m...
2,"""Tolak penundaan pemilu dan masa perpangjang j...",tolak tunda milu perpangjang jabat presiden pr...
3,#BacktoBDM @bdm2502 bersama Ulin Yusron (@ulin...,backtobdm ulin yusron bahas isuisu tunda milu ...
4,#Projo #Jokowi #3Periode \n\nSaya bilang ke te...,projo jokowi periode temanteman projo jeli jan...


# EKSTRAKSI TF-IDF

In [ ]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Pisahkan kolom fitur dan target
x = df['data_bersih']
y = df['VALUE SENTIMEN']

KeyError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(x)

TfidfVectorizer()

In [ ]:
#menampilkan fitur-fitur yang ada dalam korpus
print(tf_idf.get_feature_names_out())

['aamiin' 'abatantigaperiode' 'abdul' ... 'zul' 'zulhas' 'zulkifli']


In [ ]:
#menampilkan matriks jumlah token
#Data kemudian siap dilakukan proses permodelan (machine learning)
x_tf_idf = tf_idf.transform(x).toarray()
x_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
#Menampilkan Nilai tf-idf
data_tf_idf = pd.DataFrame(x_tf_idf, columns=tf_idf.get_feature_names_out()) 
data_tf_idf.to_csv('data_tfIdf.csv')


In [ ]:
data_tf_idf

,aamiin,abatantigaperiode,abdul,acara,acu,ada,adaahhhangel,adek,adhie,adhiemmassardi,...,yoo,youtube,youtuber,yuk,yusril,yusron,zara,zul,zulhas,zulkifli
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.54154,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np
X = np.array(data_tf_idf)
y = np.array(y)

NameError: ignored

In [ ]:
ekstraksi_fitur = SelectKBest(chi2, k=1000)
X_kbest_features = ekstraksi_fitur.fit_transform(X, y)
#mengurangi nilai fitur
print('nilai_asli feature number:', X.shape[1])
print('nilai yang dikurangi feature number:', X_kbest_features.shape[1])

ValueError: ignored

In [ ]:
data_tf_idf = pd.DataFrame(ekstraksi_fitur.scores_, columns=['nilai']) 
data_tf_idf

In [ ]:
feature = tf_idf.get_feature_names_out() 
data_tf_idf['fitur'] = feature 
data_tf_idf

In [ ]:
data_tf_idf.sort_values(by='nilai', ascending=False)

In [ ]:
mask_data = ekstraksi_fitur.get_support()
mask_data

array([ True, False,  True, ..., False,  True,  True])

In [ ]:
new_feature = []

for bool, f in zip(mask_data, feature):
  if bool:
    new_feature.append(f)
selected_feature = new_feature
selected_feature

In [ ]:
kbest_feature = {} # Buat kamus kosong
for (k,v) in tf_idf.vocabulary_.items():
   if k in selected_feature:
    kbest_feature[k] = v

In [ ]:
kbest_feature

#Perhitungan Multinomial Naive Bayes

In [ ]:
#Mengimport library multinomial naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from joblib import dump

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.1, random_state=10)

In [ ]:
#data training
#Training the model
algorithm = MultinomialNB()
model = algorithm.fit(X_train, y_train)
# Simpan model hasil traning
dump(model, filename='model_1.joblib')

['model_1.joblib']

In [ ]:
#menggunakan model yang telah dilatih untuk prediksi label data uji
model_pred = model.predict(x_test)

In [ ]:
#menampilkan hasil prediksi model
model_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0])

In [ ]:
#menampilkan hasil prediksi label
y_test

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0])

# EVALUASI MODEL

In [ ]:
prediksi_true = (model_pred == y_test).sum()
prediksi_false = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:',prediksi_true)
print('Jumlah prediksi salah\t:',prediksi_false)

akurasi = prediksi_true / (prediksi_true + prediksi_false)*100
print('akurasi pengujian\t:',akurasi, '%')

Jumlah prediksi benar	: 94
Jumlah prediksi salah	: 38
akurasi pengujian	: 71.21212121212122 %


In [ ]:
from sklearn.metrics import confusion_matrix

uji = confusion_matrix(y_test, model_pred)
print("Confussion Matrix:\n",uji)

Confussion Matrix:
 [[37 28]
 [10 57]]


In [ ]:
from sklearn.metrics import classification_report

print('Hasil Klasifikasi:\n ',classification_report(y_test, model_pred))

Hasil Klasifikasi:
                precision    recall  f1-score   support

           0       0.79      0.57      0.66        65
           1       0.67      0.85      0.75        67

    accuracy                           0.71       132
   macro avg       0.73      0.71      0.71       132
weighted avg       0.73      0.71      0.71       132



# Hasil Prediksi dokumen yang diinpuntkan...

In [ ]:
vocab = selected_feature
input_text = input("Masukkan Opini yang berkaitan dengan masa jabatan presiden 3 periode: ")
#melakukan proses preprocessing
pre_input_text = text_preprocessing_proses(input_text)
#mendenifisikasn Tf-idf
tf_idf_vector = TfidfVectorizer(vocabulary=set(vocab))
#melakukan prediksi
result = model.predict(tf_idf_vector.fit_transform([pre_input_text]))

print('\nHasil text preprocessing: ', pre_input_text)

if(result==1):
  tweet = "Positif"
else:
  tweet = "Negatif"

print('\nHasil prediksi:',input_text,'\nadalah sentimen',tweet)

Masukkan Opini yang berkaitan dengan masa jabatan presiden 3 periode: jokowi tolak tiga periode

Hasil text preprocessing:  jokowi tolak tiga periode

Hasil prediksi: jokowi tolak tiga periode 
adalah sentimen Positif
